# GS Loader

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import datetime as dt
import xlrd
import os
import json
os.getcwd()
print('working in directory {}.'.format(os.getcwd()))

working in directory /home/jupyter-stevenhurwitt/la-tools-test/GS.


## GS Template

In [2]:
gs_template = pd.read_excel('GS_template.xls')
gs_template.head()

,MARKET,DISCO_CODE,LDC_ACCOUNT_NUMBER,IDR_SCALAR,RATE_CLASS,RATE_SUBCLASS,LOSS_CLASS,ZONE,BUS,STRATA_VARIABLE,...,INTERMEDIATE USAGE,TOTAL USAGE_KWH,USAGE UOM,METER READ TYPE,ON-PEAK DEMAND,OFF-PEAK DEMAND,INTERMEDIATE DEMAND,TOTAL DEMAND_KW,DEMAND UOM,CURRENT_RATE_OPTION
0,NaN,NaN,NaN,SCALAR,NaN,NaN,NaN,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
1,NaN,NaN,NaN,SCALAR,NaN,NaN,NaN,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
2,NaN,NaN,NaN,SCALAR,NaN,NaN,NaN,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
3,NaN,NaN,NaN,SCALAR,NaN,NaN,NaN,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
4,NaN,NaN,NaN,SCALAR,NaN,NaN,NaN,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN


### Attributes

In [3]:
n = gs_template.shape[0]
market = 'NEPOOL'
disco = 'MECO'
ldc = 123456789
idr = False
rate = 'G2'
loss = 'G2'
profile = 'G2'
mrc = 10
cap_tag = 15.87
tpt = 0

fname = ''.join(['GS_', market, '_', disco, '_', str(ldc), '.xls'])

In [4]:
print('writing file as: {}.'.format(fname))

writing file as: GS_NEPOOL_MECO_123456789.xls.


## Today

In [5]:
today = dt.datetime.today()

month = today.month
day = today.day
year = today.year

stop_dates = []
(month, day, year)

(1, 23, 2020)

## Generate last year

In [6]:
if day >= mrc:
    last_read = ''.join(['1/', str(mrc), '/20'])
    last_date = dt.datetime.strptime(last_read, '%m/%d/%y')
    
last_date
stop_dates.append(last_date)

In [7]:
for i in range(n):
    next_read = stop_dates[i] - dt.timedelta(days = 30)
    stop_dates.append(next_read)

In [8]:
stop_dates

[datetime.datetime(2020, 1, 10, 0, 0),
 datetime.datetime(2019, 12, 11, 0, 0),
 datetime.datetime(2019, 11, 11, 0, 0),
 datetime.datetime(2019, 10, 12, 0, 0),
 datetime.datetime(2019, 9, 12, 0, 0),
 datetime.datetime(2019, 8, 13, 0, 0),
 datetime.datetime(2019, 7, 14, 0, 0),
 datetime.datetime(2019, 6, 14, 0, 0),
 datetime.datetime(2019, 5, 15, 0, 0),
 datetime.datetime(2019, 4, 15, 0, 0),
 datetime.datetime(2019, 3, 16, 0, 0),
 datetime.datetime(2019, 2, 14, 0, 0),
 datetime.datetime(2019, 1, 15, 0, 0)]

# Fill Attributes

In [9]:
print(gs_template.head())

   MARKET  DISCO_CODE  LDC_ACCOUNT_NUMBER IDR_SCALAR  RATE_CLASS  \
0     NaN         NaN                 NaN     SCALAR         NaN   
1     NaN         NaN                 NaN     SCALAR         NaN   
2     NaN         NaN                 NaN     SCALAR         NaN   
3     NaN         NaN                 NaN     SCALAR         NaN   
4     NaN         NaN                 NaN     SCALAR         NaN   

   RATE_SUBCLASS  LOSS_CLASS  ZONE  BUS  STRATA_VARIABLE  ...  \
0            NaN         NaN  SEMA  NaN              NaN  ...   
1            NaN         NaN  SEMA  NaN              NaN  ...   
2            NaN         NaN  SEMA  NaN              NaN  ...   
3            NaN         NaN  SEMA  NaN              NaN  ...   
4            NaN         NaN  SEMA  NaN              NaN  ...   

   INTERMEDIATE USAGE  TOTAL USAGE_KWH USAGE UOM METER READ TYPE  \
0                 NaN              NaN     KWH\n               H   
1                 NaN              NaN     KWH\n               H

## Attributes

In [10]:
list(gs_template.columns)

['MARKET',
 'DISCO_CODE',
 'LDC_ACCOUNT_NUMBER',
 'IDR_SCALAR',
 'RATE_CLASS',
 'RATE_SUBCLASS',
 'LOSS_CLASS',
 'ZONE',
 'BUS',
 'STRATA_VARIABLE',
 'PROFILE_CLASS',
 'PREMISE_NAME',
 'PREMISE_ADDRESS',
 'PREMISE_ADDRESS2',
 'PREMISE_CITY',
 'PREMISE_STATE',
 'PREMISE_ZIP',
 'PREMISE_ZIP+4',
 'COUNTY',
 'COUNTRY',
 'TPT_LOAD',
 'ICAP_TAG',
 'METER SERIAL NO.',
 'METER READ CYCLE',
 'START_DATE',
 'STOP_DATE',
 'ON-PEAK USAGE',
 'OFF-PEAK USAGE',
 'INTERMEDIATE USAGE',
 'TOTAL USAGE_KWH',
 'USAGE UOM',
 'METER READ TYPE',
 'ON-PEAK DEMAND',
 'OFF-PEAK DEMAND',
 'INTERMEDIATE DEMAND',
 'TOTAL DEMAND_KW',
 'DEMAND UOM',
 'CURRENT_RATE_OPTION']

## Fill

In [11]:
gs_template['MARKET'] = market
gs_template['DISCO_CODE'] = disco
gs_template['LDC_ACCOUNT_NUMBER'] = ldc
gs_template['IDR_SCALAR'] = True
gs_template['RATE_CLASS'] = rate
gs_template['RATE_SUBCLASS']
gs_template['LOSS_CLASS'] = loss
gs_template['ZONE']
gs_template['BUS']
gs_template['STRATA_VARIABLE']
gs_template['PROFILE_CLASS'] = profile
gs_template['PREMISE_NAME']
gs_template['PREMISE_ADDRESS']
gs_template['PREMISE_CITY']
gs_template['PREMISE_STATE']
gs_template['PREMISE_ZIP']
gs_template['PREMISE_ZIP+4']
gs_template['COUNTY']
gs_template['COUNTRY']
gs_template['TPT_LOAD'] = tpt
gs_template['ICAP_TAG'] = cap_tag
gs_template['METER SERIAL NUMBER']
gs_template['METER READ CYCLE'] = mrc
gs_template['START_DATE']
gs_template['STOP_DATE'] = stop_dates
gs_template['ON-PEAK USAGE']
gs_template['OFF-PEAK USAGE']
gs_template['INTERMEDIATE USAGE']
gs_template['TOTAL USAGE_KW']
gs_template['USAGE UOM']
gs_template['METER READ TYPE']
gs_template['ON-PEAK DEMAND']
gs_template['OFF-PEAK DEMAND']
gs_template['INTERMEDIATE DEMAND']
gs_template['TOTAL DEMAND_KW']
gs_template['DEMAND UOM']
gs_template['CURRENT_RATE_OPTION']

KeyError: 'METER SERIAL NUMBER'

# GS File

In [12]:
gs_template.head()

,MARKET,DISCO_CODE,LDC_ACCOUNT_NUMBER,IDR_SCALAR,RATE_CLASS,RATE_SUBCLASS,LOSS_CLASS,ZONE,BUS,STRATA_VARIABLE,...,INTERMEDIATE USAGE,TOTAL USAGE_KWH,USAGE UOM,METER READ TYPE,ON-PEAK DEMAND,OFF-PEAK DEMAND,INTERMEDIATE DEMAND,TOTAL DEMAND_KW,DEMAND UOM,CURRENT_RATE_OPTION
0,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
1,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
2,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
3,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
4,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN


In [13]:
gs_template.tail()

,MARKET,DISCO_CODE,LDC_ACCOUNT_NUMBER,IDR_SCALAR,RATE_CLASS,RATE_SUBCLASS,LOSS_CLASS,ZONE,BUS,STRATA_VARIABLE,...,INTERMEDIATE USAGE,TOTAL USAGE_KWH,USAGE UOM,METER READ TYPE,ON-PEAK DEMAND,OFF-PEAK DEMAND,INTERMEDIATE DEMAND,TOTAL DEMAND_KW,DEMAND UOM,CURRENT_RATE_OPTION
7,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
8,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
9,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
10,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN
11,NEPOOL,MECO,123456789,True,G2,NaN,G2,SEMA,NaN,NaN,...,NaN,NaN,KWH\n,H,NaN,NaN,NaN,0,KW\n,NaN


In [14]:
fname

'GS_NEPOOL_MECO_123456789.xls'

In [17]:
gs_template.to_excel(fname)
print('wrote file as {}.'.format(fname))

wrote file as GS_NEPOOL_MECO_123456789.xls.
